# Note, a geojson is needed for Folium. Json can be found ===> [here:](https://github.com/python-visualization/folium/blob/master/examples/data/world-countries.json) <====

In [1]:
# Needed import
import folium
from folium import plugins
import json
import pandas as pd
import geopandas
import math
import numpy as np
import base64
import matplotlib.pyplot as plt
from folium import IFrame

In [2]:
# Any df with those following columns should be fine, as an example, you can take those
df_final = pd.read_pickle("./data/final_df.pkl")
df_final['profit'] = df_final['export_v'] - df_final['import_v']
df_final['diff_quantity'] = df_final['export_q'] - df_final['import_q']
df_final.head(5)

,area_code,area_crops,item_crops,year,item_trades,area_harvested,production,yield,export_q,export_v,import_q,import_v,hs12_code,parent_group,child_group,parent_description,child_description,ISO3 Code,profit,diff_quantity
0,1,Armenia,Wheat,1992,Wheat,65500.0,141483.0,21600.0,NaN,NaN,400000.0,60000.0,100111,10,1001,Cereals,Wheat and meslin,ARM,NaN,NaN
1,1,Armenia,Wheat,1993,Wheat,97900.0,217900.0,22257.0,NaN,NaN,408000.0,59000.0,100111,10,1001,Cereals,Wheat and meslin,ARM,NaN,NaN
2,1,Armenia,Wheat,1993,"Flour, wheat",97900.0,217900.0,22257.0,NaN,NaN,46000.0,9400.0,100111,10,1001,Cereals,Wheat and meslin,ARM,NaN,NaN
3,1,Armenia,Wheat,1994,Wheat,85697.0,152900.0,17842.0,NaN,NaN,327000.0,52000.0,100111,10,1001,Cereals,Wheat and meslin,ARM,NaN,NaN
4,1,Armenia,Wheat,1994,"Flour, wheat",85697.0,152900.0,17842.0,NaN,NaN,55000.0,14700.0,100111,10,1001,Cereals,Wheat and meslin,ARM,NaN,NaN


### Below is the main function.
Example of how to use it follows in the next cell.

In [3]:
def show_production_all_countries(df, year, products=None, coloring='profit', filename='data/countries_profit.html'):
    """
    Complete function that takes some predefined df and output a map of the requested food for a given year.
    Show a folium map and then save the plot in an html file
    """
    
    df = df[df['year'] == year]
    
    if products == None:
        products = df.item_trades[:10]
    
    
    #layer_types = ['value', 'quantity']
    m = folium.Map([40,0], zoom_start=2)
    #fg = folium.FeatureGroup(overlay=True, name='groups', control=False)
    #m.add_child(fg)
    
    by_product = df.groupby('item_trades')
    i = 0
    for product in products:
        df_product = by_product.get_group(product)
        min_profit = df_product[coloring].min()
        max_profit = df_product[coloring].max()

        countries = geopandas.read_file('data/world-countries.json')
        #cm = countries.merge(df, left_on='id', right_on='iso3_code')
        cm = countries.merge(df_product, left_on='id', right_on='ISO3 Code')
        
        cm['profit'] = cm['profit'].apply(lambda value : 0 if math.isnan(value) else f'{int(value):,}')
        cm['export_v'] = cm['export_v'].apply(lambda value : 0 if math.isnan(value) else f'{int(value):,}')
        cm['import_v'] = cm['import_v'].apply(lambda value : 0 if math.isnan(value) else f'{int(value):,}')
        
        cm['diff_quantity'] = cm['diff_quantity'].apply(lambda value : 0 if math.isnan(value) else f'{int(value):,}')
        cm['export_q'] = cm['export_q'].apply(lambda value : 0 if math.isnan(value) else f'{int(value):,}')
        cm['import_q'] = cm['import_q'].apply(lambda value : 0 if math.isnan(value) else f'{int(value):,}')
        
        
        cm = cm[['id', 'name', 'geometry', 'item_trades', 'export_q', 'import_q', 'export_v', 'import_v', 'ISO3 Code', 'profit', 'diff_quantity']]
        cm.to_file("data/countries-production.json", driver='GeoJSON')
        geo_json_data = json.load(open('data/countries-production.json'))

        def my_color_function(feature):
            """Maps high values to green and negative values to red."""
            if len(df[df['ISO3 Code'] == feature['properties']['id']][coloring]) == 0:
                return '#ffffff'
            value = df[(df['ISO3 Code'] == feature['properties']['id'])&(df['item_trades'] == feature['properties']['item_trades'])][coloring].iloc[0]
            
            if math.isnan(value):
                return "#000000"
            if value > 0:
                green = hex(127 + int((value / max_profit) * 255 / 2 ))[-2:]
                return '#00' + green + '00'
            else:
                red = hex(int(127 + (value / min_profit) * 255 / 2 ))[-2:]
                return '#' + red + '00' + '00'

        def my_highlight_function(feature):
            return {
                'fillColor': 'black',
                'color' : 'black',
                'weight' : 3,
                'dashArray' : '5, 5'
                }


        
        layer = folium.GeoJson(
            geo_json_data,
            style_function=lambda feature: {
                'fillColor': my_color_function(feature),
                'color' : 'black',
                'weight' : 1.5,
                'dashArray' : '5, 5'
                },
            overlay=True,
            show=False,
            highlight_function=my_highlight_function,
            tooltip=folium.features.GeoJsonTooltip(
                fields=['name', 'profit', 'export_v', 'import_v', 'diff_quantity', 'export_q', 'import_q'],
                aliases=['State name:',  'Gross Profit:', 'Export value:', 'Import value:', 'Difference of quantity:', 'Export quantity:', 'Import quantity:'],
                )
            )
        #g = plugins.FeatureGroupSubGroup(fg, coloring, show=False)
        #m.add_child(g)
        layer.layer_name = product
        layer.add_to(m)
        #layer.add_to(fg)
        
    plugins.Fullscreen(
        position='bottomright',
        title='Expand me',
        title_cancel='Exit me',
        force_separate_button=True
    ).add_to(m)
       
    folium.LayerControl().add_to(m)
    
    m.save(filename)
    return m

In [4]:
# Example: Show a specific list of product, in year 2017. Color in function of "diff_quantity" column
show_production_all_countries(df_final, 2017, ['Maté', 'Avocados', 'Pistachios', 'Coffee, green', 'Cereals, breakfast'], coloring='diff_quantity')

In [5]:
# Example: Show other features in 1987, color by Gross profit (Profit brut)
show_production_all_countries(df_final, 1987, ['Maté', 'Avocados', 'Pistachios', 'Coffee, green', 'Cereals, breakfast', 'Wheat'], coloring='profit')